In [1]:
# To do 
# Use one of datasets here https://www.ibm.com/communities/analytics/watson-analytics-blog/guide-to-sample-datasets/ and do same 
# First identity the types of columns (Yes/No, list)
# Arrange the columns in the vector of yes/no, List
# Null should be reaplced with something 

# Goal of this model is to predict Customer Churn using RandomForest
# Data is taken from https://www.ibm.com/communities/analytics/watson-analytics-blog/guide-to-sample-datasets/
from pyspark.sql import SQLContext

from pyspark import SparkContext

sqlContext = SQLContext(sc)

df = sqlContext.read.csv('/FileStore/tables/WA_Fn_UseC__Telco_Customer_Churn-89c80.csv',header='true', inferSchema='true')
df.cache()

Out[ 1 ]: DataFrame[customerID: string, gender: string, SeniorCitizen: int, Partner: string, Dependents: string, tenure: int, PhoneService: string, MultipleLines: string, InternetService: string, OnlineSecurity: string, OnlineBackup: string, DeviceProtection: string, TechSupport: string, StreamingTV: string, StreamingMovies: string, Contract: string, PaperlessBilling: string, PaymentMethod: string, MonthlyCharges: double, TotalCharges: double, Churn: string]

In [2]:
# Step 1 - Data Analysis
# Know more about data
# Display the distinct values for Churn and their respective counts

df.select("Churn").distinct().show()
df.groupBy("Churn").count().show()

+-----+
Churn|
+-----+
 No|
 Yes|
+-----+

+-----+-----+
Churn|count|
+-----+-----+
 No| 5174|
 Yes| 1869|
+-----+-----+

In [3]:
# This step will replace blank values with zero, if any
# or drop the rows with null values.
df = df.replace('', "0")
# drop rows with null values
df = df.na.drop()
 

In [4]:
df.count() 

Out[ 4 ]: 7043

In [5]:
df.show(5)

+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService| MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies| Contract|PaperlessBilling| PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
7590-VHVEG|Female| 0| Yes| No| 1| No|No phone service| DSL| No| Yes| No| No| No| No|Month-to-month| Yes| Electronic check| 29.85| 29.85| No|
5575-GNVDE| Male| 0| No| No| 34| Yes| No| DSL| Yes| No| Yes| No| No| No| One year| No| Mailed check| 56.95| 1889.5| No|
3668-QPYBK| Male| 0| No| No| 2| Yes| No| DSL| Yes| Yes| No| No| No| No|Month-to-month| Yes| Mailed check| 53.85| 108.15| Yes|
7795-CFOCW| Male| 0| No| No| 45| No|No phone service| DSL| Yes| No| Yes| Yes| No| No| One year| No|Bank transfer (au...| 42.3| 1840.75| No|
9237-HQITU|Female| 0| No| No| 2| Yes| No| Fiber optic| No| No| No| No| No| No|Month-to-month| Yes| Electronic check| 70.7| 151.65| Yes|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
only showing top 5 rows

In [6]:
# In the data few columns has other values than Yes/No, replace those values
# it can be done as bulk way also
from pyspark.sql.functions import *

df = df.withColumn("MultipleLines", regexp_replace('MultipleLines', 'No phone service', 'No') )

df = df.withColumn("OnlineSecurity", regexp_replace('OnlineSecurity', 'No internet service', 'No') )
df = df.withColumn("OnlineBackup", regexp_replace('OnlineBackup', 'No internet service', 'No') )
df = df.withColumn("StreamingMovies", regexp_replace('StreamingMovies', 'No internet service', 'No') )
df = df.withColumn("StreamingTV", regexp_replace('StreamingMovies', 'No internet service', 'No') )
df = df.withColumn("TechSupport", regexp_replace('StreamingMovies', 'No internet service', 'No') )
df = df.withColumn("DeviceProtection", regexp_replace('StreamingMovies', 'No internet service', 'No') )




In [7]:
df.select("OnlineSecurity").distinct().show()

+--------------+
OnlineSecurity|
+--------------+
 No|
 Yes|
+--------------+

In [8]:
# Drop the unwanted column
df = df.drop("customerID")
df = df.drop("tenure")
df.show(5)


+------+-------------+-------+----------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
gender|SeniorCitizen|Partner|Dependents|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies| Contract|PaperlessBilling| PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+------+-------------+-------+----------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
Female| 0| Yes| No| No| No| DSL| No| Yes| No| No| No| No|Month-to-month| Yes| Electronic check| 29.85| 29.85| No|
 Male| 0| No| No| Yes| No| DSL| Yes| No| No| No| No| No| One year| No| Mailed check| 56.95| 1889.5| No|
 Male| 0| No| No| Yes| No| DSL| Yes| Yes| No| No| No| No|Month-to-month| Yes| Mailed check| 53.85| 108.15| Yes|
 Male| 0| No| No| No| No| DSL| Yes| No| No| No| No| No| One year| No|Bank transfer (au...| 42.3| 1840.75| No|
Female| 0| No| No| Yes| No| Fiber optic| No| No| No| No| No| No|Month-to-month| Yes| Electronic check| 70.7| 151.65| Yes|
+------+-------------+-------+----------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
only showing top 5 rows

In [9]:
# This function is to replace Yes/No/other values as 0 or 1.
recode_dictionary = {
    'YNU': {
        'Yes': 1,
        'No': 0,
        'U': 0
    }
}
import pyspark.sql.types as typ

# Find out Yes/No Columns

# find out y/N/U columns
cols = [(col.name, col.dataType) for col in df.schema]

YNU_cols = []

for i, s in enumerate(cols):
    if s[1] == typ.StringType():
        dis = df.select(s[0]) \
            .distinct() \
            .rdd \
            .map(lambda row: row[0]) \
            .collect()

        if 'Yes' in dis:
            YNU_cols.append(s[0])

In [10]:
print(YNU_cols)

['Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 'Churn']

In [11]:
df.select("DeviceProtection").distinct().show()

+----------------+
DeviceProtection|
+----------------+
 No|
 Yes|
+----------------+

In [12]:
# now transform the YNU cols to 0/1
import pyspark.sql.functions as func
def recode(col, key):        
    return recode_dictionary[key][col] 
  
rec_integer = func.udf(recode, typ.IntegerType())

exprs_YNU = [
    rec_integer(x, func.lit('YNU')).alias(x) 
    if x in YNU_cols 
    else x 
    for x in df.columns
]

In [13]:
exprs_YNU

Out[ 13 ]: 
['gender',
 'SeniorCitizen',
 Column<recode(Partner, YNU) AS `Partner`>,
 Column<recode(Dependents, YNU) AS `Dependents`>,
 Column<recode(PhoneService, YNU) AS `PhoneService`>,
 Column<recode(MultipleLines, YNU) AS `MultipleLines`>,
 'InternetService',
 Column<recode(OnlineSecurity, YNU) AS `OnlineSecurity`>,
 Column<recode(OnlineBackup, YNU) AS `OnlineBackup`>,
 Column<recode(DeviceProtection, YNU) AS `DeviceProtection`>,
 Column<recode(TechSupport, YNU) AS `TechSupport`>,
 Column<recode(StreamingTV, YNU) AS `StreamingTV`>,
 Column<recode(StreamingMovies, YNU) AS `StreamingMovies`>,
 'Contract',
 Column<recode(PaperlessBilling, YNU) AS `PaperlessBilling`>,
 'PaymentMethod',
 'MonthlyCharges',
 'TotalCharges',
 Column<recode(Churn, YNU) AS `Churn`>]

In [14]:
df = df.select(exprs_YNU)

In [15]:
df.show(5)
# InternetService is category column nd not binary values so cant be converted into Yes/No

+------+-------------+-------+----------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
gender|SeniorCitizen|Partner|Dependents|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies| Contract|PaperlessBilling| PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+------+-------------+-------+----------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
Female| 0| 1| 0| 0| 0| DSL| 0| 1| 0| 0| 0| 0|Month-to-month| 1| Electronic check| 29.85| 29.85| 0|
 Male| 0| 0| 0| 1| 0| DSL| 1| 0| 0| 0| 0| 0| One year| 0| Mailed check| 56.95| 1889.5| 0|
 Male| 0| 0| 0| 1| 0| DSL| 1| 1| 0| 0| 0| 0|Month-to-month| 1| Mailed check| 53.85| 108.15| 1|
 Male| 0| 0| 0| 0| 0| DSL| 1| 0| 0| 0| 0| 0| One year| 0|Bank transfer (au...| 42.3| 1840.75| 0|
Female| 0| 0| 0| 1| 0| Fiber optic| 0| 0| 0| 0| 0| 0|Month-to-month| 1| Electronic check| 70.7| 151.65| 1|
+------+-------------+-------+----------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
only showing top 5 rows

In [16]:
# Identify the categorical columns 
categorical_features = ['gender', 'Contract',
                      'PaymentMethod', 'InternetService' ]

In [17]:
def create_category_vars( dataset, field_name ):
  idx_col = field_name + "Index"
  col_vec = field_name + "Vec"

  month_stringIndexer = StringIndexer( inputCol=field_name,
                                       outputCol=idx_col )

  month_model = month_stringIndexer.fit( dataset )
 #   It will replace the string values with numbers for particular column 
  
  month_indexed = month_model.transform( dataset )

  month_encoder = OneHotEncoder( dropLast=True,
                                 inputCol=idx_col,
                                 outputCol= col_vec )

  return month_encoder.transform( month_indexed )

In [18]:
# OneHot Encoding for all the categorical columns

from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, PolynomialExpansion, VectorIndexer

for col in categorical_features:
  df = create_category_vars( df, col )

In [19]:
df.show(5)

+------+-------------+-------+----------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+-----------+-------------+-------------+-------------+------------------+----------------+--------------------+------------------+
gender|SeniorCitizen|Partner|Dependents|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies| Contract|PaperlessBilling| PaymentMethod|MonthlyCharges|TotalCharges|Churn|genderIndex| genderVec|ContractIndex| ContractVec|PaymentMethodIndex|PaymentMethodVec|InternetServiceIndex|InternetServiceVec|
+------+-------------+-------+----------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+-----------+-------------+-------------+-------------+------------------+----------------+--------------------+------------------+
Female| 0| 1| 0| 0| 0| DSL| 0| 1| 0| 0| 0| 0|Month-to-month| 1| Electronic check| 29.85| 29.85| 0| 1.0| (1,[],[])| 0.0|(2,[0],[1.0])| 0.0| (3,[0],[1.0])| 1.0| (2,[1],[1.0])|
 Male| 0| 0| 0| 1| 0| DSL| 1| 0| 0| 0| 0| 0| One year| 0| Mailed check| 56.95| 1889.5| 0| 0.0|(1,[0],[1.0])| 2.0| (2,[],[])| 1.0| (3,[1],[1.0])| 1.0| (2,[1],[1.0])|
 Male| 0| 0| 0| 1| 0| DSL| 1| 1| 0| 0| 0| 0|Month-to-month| 1| Mailed check| 53.85| 108.15| 1| 0.0|(1,[0],[1.0])| 0.0|(2,[0],[1.0])| 1.0| (3,[1],[1.0])| 1.0| (2,[1],[1.0])|
 Male| 0| 0| 0| 0| 0| DSL| 1| 0| 0| 0| 0| 0| One year| 0|Bank transfer (au...| 42.3| 1840.75| 0| 0.0|(1,[0],[1.0])| 2.0| (2,[],[])| 2.0| (3,[2],[1.0])| 1.0| (2,[1],[1.0])|
Female| 0| 0| 0| 1| 0| Fiber optic| 0| 0| 0| 0| 0| 0|Month-to-month| 1| Electronic check| 70.7| 151.65| 1| 1.0| (1,[],[])| 0.0|(2,[0],[1.0])| 0.0| (3,[0],[1.0])| 0.0| (2,[0],[1.0])|
+------+-------------+-------+----------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+-----------+-------------+-------------+-------------+------------------+----------------+--------------------+------------------+
only showing top 5 rows

In [20]:
# Get all the YNU columns
YNU_cols_updated =  ['Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling']

In [21]:
# create features for vector Assembler - Columns being supplied as a vector 
featureCols = YNU_cols_updated + ['genderVec', 
                                  'ContractVec',
                                  'PaymentMethodVec', 
                                  'InternetServiceVec']   + ['MonthlyCharges','TotalCharges' ]

In [22]:
df = df.select('Partner',
 'Dependents',
 'PhoneService',
 'MultipleLines',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'PaperlessBilling',
 'genderVec',
 'ContractVec',
 'PaymentMethodVec',
 'InternetServiceVec', df.MonthlyCharges.cast("float"), df.TotalCharges.cast("float"), df.Churn)

df = df.withColumnRenamed("Churn", "label")

In [23]:
# create VectorAssembler to convert the features into vector

assembler = VectorAssembler( inputCols = featureCols, outputCol = "features")

# Combine both vectors into single vector to be processed by an algorithm
customer_train_df = assembler.transform( df )



In [24]:
customer_train_df.printSchema()


root
-- Partner: integer (nullable = true)
-- Dependents: integer (nullable = true)
-- PhoneService: integer (nullable = true)
-- MultipleLines: integer (nullable = true)
-- OnlineSecurity: integer (nullable = true)
-- OnlineBackup: integer (nullable = true)
-- DeviceProtection: integer (nullable = true)
-- TechSupport: integer (nullable = true)
-- StreamingTV: integer (nullable = true)
-- StreamingMovies: integer (nullable = true)
-- PaperlessBilling: integer (nullable = true)
-- genderVec: vector (nullable = true)
-- ContractVec: vector (nullable = true)
-- PaymentMethodVec: vector (nullable = true)
-- InternetServiceVec: vector (nullable = true)
-- MonthlyCharges: float (nullable = true)
-- TotalCharges: float (nullable = true)
-- label: integer (nullable = true)
-- features: vector (nullable = true)

In [25]:
# Split the data in 70% & 30% 
customer_train, customer_test = customer_train_df.randomSplit([0.7, 0.3])


In [26]:
# now build the random forest model t

from pyspark.ml.classification import RandomForestClassifier

# Define the input and outut where Label is ouput and features in input vector 
rf = RandomForestClassifier(labelCol="label", featuresCol="features")
# Fitting a model in classifier 
rfModel = rf.fit(customer_train)
# Prediction is done here 
predictions = rfModel.transform(customer_test)

In [27]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model, find accuracy of the model by checking the label(churn value) & predicted value (predction) 
evaluator = BinaryClassificationEvaluator()
accuracyDt = evaluator.evaluate(predictions)
print("Test Error = %g " % (1.0 - accuracyDt))

Test Error = 0.154354

In [28]:
# Label is Churn columnsuplied as input 
selected = predictions.select("label", "prediction", "features", "MonthlyCharges")
selected.show()

+-----+----------+--------------------+--------------+
label|prediction| features|MonthlyCharges|
+-----+----------+--------------------+--------------+
 0| 0.0|(21,[15,18,19,20]...| 25.85|
 1| 0.0|(21,[12,14,18,19,...| 25.25|
 1| 0.0|(21,[12,15,18,19,...| 25.05|
 1| 0.0|(21,[12,15,18,19,...| 25.2|
 0| 0.0|(21,[12,16,18,19,...| 25.25|
 0| 0.0|(21,[11,18,19,20]...| 33.7|
 0| 0.0|(21,[11,15,18,19,...| 35.55|
 1| 0.0|(21,[11,12,14,18,...| 24.75|
 0| 0.0|(21,[11,12,14,18,...| 30.5|
 1| 0.0|(21,[10,12,18,19,...| 25.1|
 0| 0.0|(21,[10,12,18,19,...| 44.45|
 1| 0.0|(21,[10,12,14,18,...| 30.4|
 0| 0.0|(21,[10,12,14,18,...| 31.2|
 1| 1.0|(21,[10,12,14,18,...| 35.75|
 1| 1.0|(21,[10,12,14,18,...| 36.45|
 0| 0.0|(21,[10,12,14,18,...| 39.85|
 0| 0.0|(21,[10,12,15,18,...| 24.9|
 1| 0.0|(21,[10,12,15,18,...| 25.1|
 0| 0.0|(21,[10,11,12,18,...| 24.9|
 0| 0.0|(21,[10,11,12,18,...| 25.25|
+-----+----------+--------------------+--------------+
only showing top 20 rows

In [29]:
from pyspark.ml.classification import DecisionTreeClassifier

# Create initial Decision Tree Model
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxDepth=3)

# Train model with Training Data
dtModel = dt.fit(customer_train) 
# Make predictions on test data using the Transformer.transform() method.
predictions_dt = dtModel.transform(customer_test)
predictions_dt.printSchema()

root
-- Partner: integer (nullable = true)
-- Dependents: integer (nullable = true)
-- PhoneService: integer (nullable = true)
-- MultipleLines: integer (nullable = true)
-- OnlineSecurity: integer (nullable = true)
-- OnlineBackup: integer (nullable = true)
-- DeviceProtection: integer (nullable = true)
-- TechSupport: integer (nullable = true)
-- StreamingTV: integer (nullable = true)
-- StreamingMovies: integer (nullable = true)
-- PaperlessBilling: integer (nullable = true)
-- genderVec: vector (nullable = true)
-- ContractVec: vector (nullable = true)
-- PaymentMethodVec: vector (nullable = true)
-- InternetServiceVec: vector (nullable = true)
-- MonthlyCharges: float (nullable = true)
-- TotalCharges: float (nullable = true)
-- label: integer (nullable = true)
-- features: vector (nullable = true)
-- rawPrediction: vector (nullable = true)
-- probability: vector (nullable = true)
-- prediction: double (nullable = false)

In [30]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator()
accuracyDt = evaluator.evaluate(predictions_dt)
print("Test Error = %g " % (1.0 - accuracyDt))


Test Error = 0.179093

In [31]:
# now do the logistic regression  
from pyspark.mllib.classification  import LogisticRegressionWithLBFGS
LR_Model_2 = LogisticRegressionWithLBFGS \
    .train(customer_train, iterations=10)

LR_results_2 = (
        topFeatures_test.map(lambda row: row.label) \
        .zip(LR_Model_2 \
             .predict(customer_test \
                      .map(lambda row: row.features)))
    ).map(lambda row: (row[0], row[1] * 1.0))

LR_evaluation = ev.BinaryClassificationMetrics(LR_results_2)

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-1493254236634501> in <module> () 
 1 # now do the logistic regression 
 2 from pyspark . mllib . classification import LogisticRegressionWithLBFGS 
 ----> 3 LR_Model_2 = LogisticRegressionWithLBFGS . train ( customer_train , iterations = 10 ) 
 4 
 5 LR_results_2 = (

 /databricks/spark/python/pyspark/mllib/classification.pyc in train (cls, data, iterations, initialWeights, regParam, regType, intercept, corrections, tolerance, validateData, numClasses) 
 396 else : 
 397 initialWeights = [ 0.0 ] * len ( data . first ( ) . features ) * ( numClasses - 1 ) 
 --> 398 return _regression_train_wrapper ( train , LogisticRegressionModel , data , initialWeights ) 
 399 
 400 

 /databricks/spark/python/pyspark/mllib/regression.pyc in _regression_train_wrapper (train_func, modelClass, data, initial_weights) 
 208 first = data . first ( ) 
 209 if not isinstance ( first , LabeledPoint ) : 
 --> 210 raise TypeError ( "data should be an RDD of LabeledPoint, but got %s" % type ( first ) ) 
 211 if initial_weights is None : 
 212 initial_weights = [ 0.0 ] * len ( data . first ( ) . features ) 

 TypeError : data should be an RDD of LabeledPoint, but got <class 'pyspark.sql.types.Row'>

In [32]:
print('Area under PR: {0:.2f}' \
      .format(LR_evaluation.areaUnderPR))
print('Area under ROC: {0:.2f}' \
      .format(LR_evaluation.areaUnderROC))

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1493254236634502> in <module> () 
 ----> 1 print ( 'Area under PR: {0:.2f}' . format ( LR_evaluation . areaUnderPR ) ) 
 2 print ( 'Area under ROC: {0:.2f}' . format ( LR_evaluation . areaUnderROC ) ) 

 NameError : name 'LR_evaluation' is not defined